In [1]:
!pip install accelerate

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple


In [3]:
from modelscope import snapshot_download
model_dir = snapshot_download("ZhipuAI/glm-4-9b-chat-1m", cache_dir="/text2CQL")

2024-06-14 15:24:01,035 - modelscope - INFO - PyTorch version 2.3.0 Found.
2024-06-14 15:24:01,036 - modelscope - INFO - Loading ast index from /home/abc/.cache/modelscope/ast_indexer
2024-06-14 15:24:01,036 - modelscope - INFO - No valid ast index found from /home/abc/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-06-14 15:24:01,063 - modelscope - INFO - Loading done! Current index file version is 1.15.0, with md5 b6bd21a43433c46216df735a83175175 and a total number of 980 components indexed
Downloading: 100%|██████████| 1.38k/1.38k [00:00<00:00, 4.73kB/s]
Downloading: 100%|██████████| 2.21k/2.21k [00:00<00:00, 8.62kB/s]
Downloading: 100%|██████████| 6.34k/6.34k [00:00<00:00, 25.7kB/s]
Downloading:   9%|▉         | 171M/1.82G [02:49<28:11, 1.05MB/s]  2024-06-14 15:26:58,479 - modelscope - WARNING - Downloading: /text2CQL/._____temp/ZhipuAI/glm-4-9b-chat-1m/model-00001-of-00010.safetensors failed, reason: ("Connection broken: ConnectionResetError(104, 'Connectio

In [2]:
!pip install modelscope

Looking in indexes: https://pypi.tuna.tsinghua.edu.cn/simple
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 743.0 kB/s eta 0:00:00m eta 0:00:010:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 698.4 kB/s eta 0:00:001m706.1 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.2/43.2 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.4/283.4 kB 430.8 kB/s eta 0:00:00 kB/s eta 0:00:01:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.5/34.5 MB 1.0 MB/s eta 0:00:00m eta 0:00:010:00:02m
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.6/136.6 kB 984.5 kB/s eta 0:00:00 MB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.7/254.7 kB 750.5 kB/s eta 0:00:00 kB/s eta 0:00:01:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.9/170.9 kB 803.8 kB/s eta 0:00:00 kB/s eta 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.7/89.7 kB 1.3 MB/s eta 0:00:00 MB/s eta 0:00:01
  Preparing me

In [2]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer

device = "cuda"

tokenizer = AutoTokenizer.from_pretrained("/root/autodl-tmp/glm-4-9b-chat-1m",trust_remote_code=True)


model = AutoModelForCausalLM.from_pretrained(
    "/root/autodl-tmp/glm-4-9b-chat-1m",
    torch_dtype=torch.bfloat16,
    low_cpu_mem_usage=True,
    trust_remote_code=True
).to(device).eval()


Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Loading checkpoint shards:   0%|          | 0/10 [00:00<?, ?it/s]

In [2]:
query = "给你一个query：'2012冲绳国际儿童青少年戏剧艺术节的举办时间是在哪一年'， 再给你一个list：['描述', '中文名称', '主办方', '地区', '时间', '标签'],你从list中选两个和query关系最接近的两个词，只返回你选的词，不要生成其他的任何话!"

inputs = tokenizer.apply_chat_template([{"role": "user", "content": query}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       )

inputs = inputs.to(device)

gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
with torch.no_grad():
    outputs = model.generate(**inputs, **gen_kwargs)
    outputs = outputs[:, inputs['input_ids'].shape[1]:]
    print(tokenizer.decode(outputs[0], skip_special_tokens=True))


时间,地区


In [6]:
import json
query_entity_nearest = {}
with open("blank_cypher_mention_entities.json", "r", encoding="utf-8") as file:
    cypher_queries = json.load(file)
for index, (key, value) in enumerate(cypher_queries.items()):
    tempList = []
    mention_to_entity = {}
    print(key, value)
    for item in value:
        if item['entities'] == [] or len(item['entities'])==1:
            mention_to_entity = {
                "mention": item['mention'],
                "entities": item['entities']
            }
        else:
            query = "给定查询：{} 和列表：{}，从列表中选择与查询最相关的词，仅返回所选词，除此之外不生成任何其他内容。".format(key, item['entities'])
            inputs = tokenizer.apply_chat_template([{"role": "user", "content": query}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       )

            inputs = inputs.to(device)
            gen_kwargs = {"max_length": 2500, "do_sample": True, "top_k": 1}
            with torch.no_grad():
                outputs = model.generate(**inputs, **gen_kwargs)
                outputs = outputs[:, inputs['input_ids'].shape[1]:]
                response = tokenizer.decode(outputs[0], skip_special_tokens=True)
                print(response)
                mention_to_entity = {
                        "mention": item['mention'],
                        "entities": [str(response.replace('\n', ''))]
                        }
        tempList.append(mention_to_entity)
    query_entity_nearest[key] = tempList
    print(tempList)

with open('most_similar_1entity.json', 'w', encoding='utf-8') as json_file:
    json.dump(query_entity_nearest, json_file, ensure_ascii=False, indent=2)


似乎有个日本军优叫乙夜，他是干哪行的？ [{'mention': '乙夜', 'entities': ['乙夜[词语]', '乙夜[日本军优]']}]

乙夜[日本军优]
[{'mention': '乙夜', 'entities': ['乙夜[日本军优]']}]
你知道全场为10.65的军事武器是什么吗？ [{'mention': '10.65', 'entities': []}]
[{'mention': '10.65', 'entities': []}]
说出由左鸿斌担任的三种职业都是什么？ [{'mention': '左鸿斌', 'entities': []}]
[{'mention': '左鸿斌', 'entities': []}]
2012冲绳国际儿童青少年戏剧艺术节的举办时间是在哪一年？ [{'mention': '2012冲绳国际儿童青少年戏剧艺术节', 'entities': ['2012冲绳国际儿童青少年戏剧艺术节']}]
[{'mention': '2012冲绳国际儿童青少年戏剧艺术节', 'entities': ['2012冲绳国际儿童青少年戏剧艺术节']}]
邻苯二甲酸二[2-乙基己基]酯在日常中，通俗的中文别名是什么？ [{'mention': '邻苯二甲酸二[2-乙基己基]酯', 'entities': []}]
[{'mention': '邻苯二甲酸二[2-乙基己基]酯', 'entities': []}]
你知道哪个学校设有轮训专业，其中2个为国家重点学科吗？能说前一个吗？ [{'mention': '轮训', 'entities': ['轮训']}]
[{'mention': '轮训', 'entities': ['轮训']}]
云艺文华的全称你知道是什么？ [{'mention': '云艺文华', 'entities': []}]
[{'mention': '云艺文华', 'entities': []}]
设有博士后流动站的院校拥有的前五个主要院系是？ [{'mention': '博士后流动站', 'entities': ['博士后科研流动站']}]
[{'mention': '博士后流动站', 'entities': ['博士后科研流动站']}]
名叫1945打飞机的涉及游戏有啥特征？ [{'mention': '1945打飞机

In [5]:
import json
query_relationships_nearest = {}
with open("entity_oneskip_relationship.json", "r", encoding="utf-8") as file:
    cypher_queries = json.load(file)
for index, (key, value) in enumerate(cypher_queries.items()):
    tempList = []
    mention_to_relationships = {}
    print(key, value)
    for item in value:
        if item['relationships'] == [] or len(item['relationships'])<=3:
            mention_to_relationships = {
                "mention": item['mention'],
                "relationships": item['relationships']
            }
        else:
            query ="你是一个识别语义相似度的专家，给你一个query：{},给你一系列词：{}，你从这些词中选三个和query语义最相关的，只返回你从这里面选的三个词，不要生成任何其他的话！不要回答query！比如query是'似乎有个日本军优叫乙夜，他是干哪行的'，一系列词list为['描述', '歧义关系', '中文名称', '身份', '职业'],那么你的返回值只能是这里面的三个词".format(key,item['relationships'])
            inputs = tokenizer.apply_chat_template([{"role": "user", "content": query}],
                                       add_generation_prompt=True,
                                       tokenize=True,
                                       return_tensors="pt",
                                       return_dict=True
                                       )

            inputs = inputs.to(device)
            gen_kwargs = {"max_length": 4000, "do_sample": True, "top_k": 1}
            with torch.no_grad():
                outputs = model.generate(**inputs, **gen_kwargs)
                outputs = outputs[:, inputs['input_ids'].shape[1]:]
                response = tokenizer.decode(outputs[0], skip_special_tokens=True)
                print(response)
            mention_to_relationships = {
            "mention": item['mention'],
            "relationships": response.replace('\n','').split(',')
            }
        tempList.append(mention_to_relationships)
    query_relationships_nearest[key] = tempList
    print(tempList)

with open('query_relationships_nearest_3.json', 'w', encoding='utf-8') as json_file:
    json.dump( query_relationships_nearest, json_file, ensure_ascii=False, indent=2)

似乎有个日本军优叫乙夜，他是干哪行的？ [{'mention': '乙夜[日本军优]', 'relationships': ['歧义关系', '中文名称', '职业', '身份', '描述']}]

职业, 身份, 描述
[{'mention': '乙夜[日本军优]', 'relationships': ['职业', ' 身份', ' 描述']}]
你知道全场为10.65的军事武器是什么吗？ [{'mention': '10.65', 'relationships': ['定价', '型宽']}]
[{'mention': '10.65', 'relationships': ['定价', '型宽']}]
说出由左鸿斌担任的三种职业都是什么？ [{'mention': '左鸿斌', 'relationships': []}]
[{'mention': '左鸿斌', 'relationships': []}]
2012冲绳国际儿童青少年戏剧艺术节的举办时间是在哪一年？ [{'mention': '2012冲绳国际儿童青少年戏剧艺术节', 'relationships': ['时间', '地区', '主办方', '标签', '中文名称', '描述']}]

时间, 地区, 描述
[{'mention': '2012冲绳国际儿童青少年戏剧艺术节', 'relationships': ['时间', ' 地区', ' 描述']}]
邻苯二甲酸二[2-乙基己基]酯在日常中，通俗的中文别名是什么？ [{'mention': '邻苯二甲酸二[2-乙基己基]酯', 'relationships': ['英文别称', '相对分子质量', '标签', '歧义关系', '分子式', 'CAS号', '中文名称', '中文别称', '描述', '外文名称']}]

中文别称, 中文名称, 英文别称
[{'mention': '邻苯二甲酸二[2-乙基己基]酯', 'relationships': ['中文别称', ' 中文名称', ' 英文别称']}]
你知道哪个学校设有轮训专业，其中2个为国家重点学科吗？能说前一个吗？ [{'mention': '轮训', 'relationships': ['标签', '描述']}]
[{'mention': '轮训', 'relationships': [